In [1]:
import os
import getpass
from typing import Annotated
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langgraph.graph import MessagesState
from langgraph.types import Send, Command
from langchain.tools import tool
from geopy.geocoders import Nominatim
from langchain.agents import tool
import requests
from typing import Dict
from langchain.tools import tool
import requests
import json

In [2]:
def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}: ")

_set_if_undefined("GROQ_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

In [3]:
llm = ChatGroq(model="openai/gpt-oss-120b")

In [5]:
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

### DEFINING THE TOOLS FOR THE RESEARCH AGENT

In [6]:
@tool("fetch_visa_info", return_direct=True)
def fetch_visa_info(passport_country_code: str, destination_country_code: str) -> str:
    """
    LangChain Tool:
    Given a passport country code (ISO Alpha-2) and destination country code,
    fetch the visa requirements info via external API.

    Example:
        Input: ("HK", "US")
        Output: JSON string containing visa information
    """
    url = f"https://rough-sun-2523.fly.dev/visa/{passport_country_code}/{destination_country_code}"
    
    try:
        response = requests.get(url, timeout=15)
        response.raise_for_status()
        data = response.json()
        return json.dumps(data, indent=4)
    except requests.exceptions.HTTPError as http_err:
        try:
            return f"HTTP error: {http_err}\nAPI Response: {response.json()}"
        except Exception:
            return f"HTTP error: {http_err}\nRaw Response: {response.text}"
    except requests.exceptions.RequestException as err:
        return f"Request error: {err}"

### BUILD THE RESEARCH AGENT

In [7]:
research_agent = create_react_agent(
    model=llm,
    tools=[fetch_visa_info],
    prompt=("Your sole purpose is to use the tools,at every single step before you figure out the solution for the user query, you have and use tools for every step for your assigned tasks"
        "You are a research agent.\n\n"
        "INSTRUCTIONS:\n"
        "- First find the country code for the provided locations by yourself.\n"
        "- Second use the fetch_visa_info_tool to get the visa details.\n"
        "- After you're done with your tasks, respond to the supervisor directly.\n"
        "- Respond ONLY with the visa info output by the fetch_visa_info tool output in string format, do NOT include ANY other text."
    ),
    name="research_agent",
)

### checking the research agent

In [8]:
for chunk in research_agent.stream(
    {"messages": [{"role": "user", "content": "i want to go to bali from villupuram"}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: research_agent
Tool Calls:
  fetch_visa_info (fc_b6b5e5f9-22b1-455b-a4bb-61ff250cb599)
 Call ID: fc_b6b5e5f9-22b1-455b-a4bb-61ff250cb599
  Args:
    destination_country_code: ID
    passport_country_code: IN


Update from node tools:


================================= Tool Message =================================
Name: fetch_visa_info

{
    "id": 15323,
    "passport": {
        "name": "India",
        "code": "IN"
    },
    "destination": {
        "name": "Indonesia",
        "code": "ID"
    },
    "dur": null,
    "category": {
        "name": "Visa On Arrival (including eTA)",
        "code": "VOA"
    },
    "last_updated": "2025-08-27T04:27:43.611Z"
}




### DEFINE THE TOOLS FOR THE WEATHER AGENT

In [6]:
@tool
def get_weather(place: str) -> dict:
    """
    Get the current weather forecast for a given place (city/town/village).
    Uses OpenStreetMap for geocoding and Open-Meteo for weather data.
    """
    # 1. Geocode the place -> lat/lon
    geo_url = "https://nominatim.openstreetmap.org/search"
    geo_params = {"q": place, "format": "json", "limit": 1}
    geo_resp = requests.get(geo_url, params=geo_params, headers={"User-Agent": "langchain-tool"})
    geo_resp.raise_for_status()
    geo_data = geo_resp.json()
    if not geo_data:
        return {"error": f"Could not find location: {place}"}
    
    lat, lon = geo_data[0]["lat"], geo_data[0]["lon"]

    # 2. Fetch weather from Open-Meteo
    weather_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "current_weather": True,
    }
    weather_resp = requests.get(weather_url, params=params)
    weather_resp.raise_for_status()
    weather_data = weather_resp.json()

    return {
        "place": place,
        "latitude": lat,
        "longitude": lon,
        "current_weather": weather_data.get("current_weather", {}),
    }

### BUILD THE WEATHER AGENT

In [10]:
weather_agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    prompt=("Your sole purpose is to use the tools,at every single step before you figure out the solution for the user query, you have and use tools for every step for your assigned tasks"
        "You are a weather data gathering agent.\n\n"
        "INSTRUCTIONS:\n"
        "- First pass the destination place name to the tool\n"
        "- Second ,get the weather details from the tool for the destination place alone.\n"
        "- After you're done with your tasks, respond to the supervisor directly.\n"
        "- Respond ONLY with the necessary weather details for a user will consider for a vacation in string format, do NOT include ANY other text."
    ),
    name="weather_agent",
)

In [11]:
for chunk in weather_agent.stream(
    {"messages": [{"role": "user", "content": "i want to go to bali from villupuram"}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: weather_agent
Tool Calls:
  get_weather (fc_80ca7707-acb1-497b-96a5-78a6c4df8803)
 Call ID: fc_80ca7707-acb1-497b-96a5-78a6c4df8803
  Args:
    place: Bali


Update from node tools:


================================= Tool Message =================================
Name: get_weather

{"place": "Bali", "latitude": "-8.2271303", "longitude": "115.1919203", "current_weather": {"time": "2025-08-27T04:15", "interval": 900, "temperature": 24.8, "windspeed": 4.3, "winddirection": 180, "is_day": 1, "weathercode": 3}}


Update from node agent:


================================== Ai Message ==================================
Name: weather_agent

Bali: 24.8 °C, wind 4.3 km/h from 180°, overcast.




### DEFINE THE TOOLS FOR THE FLIGHT AGENT

In [12]:
import requests
from langchain.tools import tool

# Obtain the access token
def get_access_token():
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    payload = {
        'grant_type': 'client_credentials',
        'client_id': os.getenv("AMADEUS_API_KEY"),
        'client_secret': os.getenv("AMADEUS_API_SECRET")
    }
    response = requests.post(url, data=payload)
    response.raise_for_status()
    return response.json()['access_token']

@tool
def get_flight_offers(departure_city: str, arrival_city: str, travel_date: str) -> dict:
    """
    Fetch available flight offers from Amadeus API for a given route and date.
    """
    access_token = get_access_token()
    url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    params = {
        'originLocationCode': departure_city,
        'destinationLocationCode': arrival_city,
        'departureDate': travel_date,
        'adults': 1,
        'max': 5  # Limit to 5 results
    }
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()


In [13]:
flight_data = get_flight_offers.invoke({
    'departure_city': 'DEL',  # Delhi
    'arrival_city': 'MAA',    # Chennai
    'travel_date': '2025-09-15'
})
print(flight_data)


{'meta': {'count': 5, 'links': {'self': 'https://test.api.amadeus.com/v2/shopping/flight-offers?originLocationCode=DEL&destinationLocationCode=MAA&departureDate=2025-09-15&adults=1&max=5'}}, 'data': [{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'isUpsellOffer': False, 'lastTicketingDate': '2025-08-27', 'lastTicketingDateTime': '2025-08-27', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT14H', 'segments': [{'departure': {'iataCode': 'DEL', 'terminal': '3', 'at': '2025-09-15T11:00:00'}, 'arrival': {'iataCode': 'JAI', 'terminal': '1', 'at': '2025-09-15T11:50:00'}, 'carrierCode': 'AI', 'number': '9729', 'aircraft': {'code': '320'}, 'operating': {'carrierCode': 'IX'}, 'duration': 'PT50M', 'id': '1', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'JAI', 'terminal': '2', 'at': '2025-09-15T22:25:00'}, 'arrival': {'iataCode': 'MAA', 'terminal': '4', 'at': '2025-09-16T01:0

In [14]:
flight_agent = create_react_agent(
    model=llm,
    tools=[get_flight_offers],
    prompt=(
        "You are a flight details gathering agent. Your sole purpose is to use the tools "
        "to gather accurate flight information before giving any answer.\n\n"
        "INSTRUCTIONS:\n"
        "- First, pass the user's origin(choose a closest possible origin from the user given location), destination, and travel date to the get_flight_offers tool.\n"
        "- Use the output of the tool to generate a concise, readable summary of flights.\n"
        "- Format each flight as a bullet point with the following details:\n"
        "    • Flight number and airline\n"
        "    • Departure and arrival times (local time)\n"
        "    • Duration and stops\n"
        "    • Price\n"
        "- Respond ONLY with the flight summary in string format; do NOT include any extra explanation or text.\n"
        "- Example format:\n"
        "    • AI439 - Air India | DEL 11:30 → MAA 14:25 | 2h 55m | Non-stop | 67.51 EUR"
    ),
    name="flight_agent",
)

In [15]:
for chunk in flight_agent.stream(
    {"messages": [{"role": "user", "content": "i want to go to bali from villupuram on 15th september."}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: flight_agent
Tool Calls:
  get_flight_offers (fc_5dd1aad9-16ca-46ff-9bd4-77d88ed2e31d)
 Call ID: fc_5dd1aad9-16ca-46ff-9bd4-77d88ed2e31d
  Args:
    arrival_city: DPS
    departure_city: MAA
    travel_date: 2025-09-15


Update from node tools:


================================= Tool Message =================================
Name: get_flight_offers

{"meta": {"count": 5, "links": {"self": "https://test.api.amadeus.com/v2/shopping/flight-offers?originLocationCode=MAA&destinationLocationCode=DPS&departureDate=2025-09-15&adults=1&max=5"}}, "data": [{"type": "flight-offer", "id": "1", "source": "GDS", "instantTicketingRequired": false, "nonHomogeneous": false, "oneWay": false, "isUpsellOffer": false, "lastTicketingDate": "2025-08-27", "lastTicketingDateTime": "2025-08-27", "numberOfBookableSeats": 9, "itineraries": [{"duration": "PT12H30M", "segments": [{"departure": {"iataCode

In [37]:
import os
import requests
from langchain_core.tools import tool

STAYAPI_KEY = os.getenv("STAYAPI_KEY")  # store your API key in env vars

@tool
def get_hotels_stayapi(
    dest_id: str,
    checkin: str,
    checkout: str,
    adults: int = 2,
    rooms: int = 1,
    children: int = 0,
    children_ages: str = "",
    rows_per_page: int = 5,
    offset: int = 0,
    language: str = "en-us",
    currency: str = "USD"
):
    """
    Fetch available hotels in a given destination with real-time availability and pricing.
    
    Args:
        dest_id: Destination ID from Booking.com (e.g., "-3233180" for Phuket)
        checkin: Check-in date in YYYY-MM-DD format
        checkout: Check-out date in YYYY-MM-DD format
        adults: Number of adults (default: 2)
        rooms: Number of rooms (default: 1)
        children: Number of children (default: 0)
        children_ages: Comma-separated ages of children (e.g., "5,10")
        rows_per_page: Number of hotels per page (default: 5)
        offset: Offset for pagination (default: 0)
        language: Language (default: "en-us")
        currency: Currency code (default: "USD")
    
    Returns:
        List of hotels with name, price, rating, and booking URL.
    """
    url = "https://api.stayapi.com/v1/booking/search"
    headers = {"x-api-key": STAYAPI_KEY}

    params = {
        "dest_id": dest_id,
        "checkin": checkin,
        "checkout": checkout,
        "adults": adults,
        "rooms": rooms,
        "children": children,
        "children_ages": children_ages,
        "rows_per_page": rows_per_page,
        "offset": offset,
        "language": language,
        "currency": currency,
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()

        if not data.get("success"):
            return {"error": data.get("message", "Unknown error from StayAPI")}

        hotels = []
        for hotel in data["data"]:
            hotels.append({
                "id": hotel["hotel_id"],
                "name": hotel["hotel_name"],
                "rating": f"{hotel['review_score']} ({hotel['review_score_word']})",
                "stars": hotel["star_rating"],
                "price": f"{hotel['min_total_price']} {hotel['currency_code']}",
                "url": hotel["url"],
                "address": hotel["address"],
                "cancellable": bool(hotel["is_free_cancellable"]),
                "prepayment": not bool(hotel["is_no_prepayment_block"])
            })

        return hotels

    except requests.RequestException as e:
        return {"error": str(e)}


In [18]:
hotel_agent = create_react_agent(
    model=llm,
    tools=[get_hotels_stayapi],
    prompt=(
        "You are a hotel details gathering agent. Your ONLY job is to use the "
        "get_hotels_rapidapi_tool to gather hotel information before giving any answer. "
        "You MUST call the tool first with the exact parameters provided by the user.\n\n"
        "TOOL PARAMETERS:\n"
        "  • city: Name of the destination city (e.g., 'Bali')\n"
        "  • check_in: Check-in date in YYYY-MM-DD format\n"
        "  • check_out: Check-out date in YYYY-MM-DD format\n"
        "  • adults: Number of adults (integer, default 2)\n\n"
        "STRICT RULES:\n"
        "1. Call get_hotels_rapidapi_tool immediately using the user-provided parameters.\n"
        "2. Only after the tool call completes, provide output.\n"
        "3. If the tool returns empty results or fails, generate fallback data, "
        "but clearly mark it as assumed.\n"
        "4. Do NOT answer or speculate before calling the tool.\n"
        "5. All output must be based on the tool results or fallback if unavailable."
    )
,
    name="hotel_agent",
)

In [19]:
for chunk in hotel_agent.stream(
    {"messages": [{"role": "user", "content": "i want to go to bali from villupuram on 15th september.we are two adults and planning to stay for 3 daysin a luxurious place."}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: hotel_agent
Tool Calls:
  get_hotels_stayapi (fc_934368c0-a92a-4d08-a124-2a9e541d9e31)
 Call ID: fc_934368c0-a92a-4d08-a124-2a9e541d9e31
  Args:
    adults: 2
    check_in: 2025-09-15
    check_out: 2025-09-18
    city: Bali


Update from node tools:


================================= Tool Message =================================
Name: get_hotels_stayapi

Error: HTTPError('422 Client Error: Unprocessable Entity for url: https://api.stayapi.com/v1/booking/search?checkin_date=2025-09-15&checkout_date=2025-09-18&adults_number=2&city_name=Bali&order_by=popularity&filter_by_currency=USD&locale=en-us&room_number=1&units=metric')
 Please fix your mistakes.


Update from node agent:


================================== Ai Message ==================================
Name: hotel_agent

**Assumed Hotel Options in Bali (Luxury Segment) – Based on typical market data (actual API call fa

In [20]:
@tool
def get_activities_opentripmap(city: str, limit: int = 5):
    """
    Fetch top tourist attractions in a city using OpenTripMap API.
    """

    # Step 1: Get city coordinates
    geo_url = "https://api.opentripmap.com/0.1/en/places/geoname"
    geo_params = {"name": city, "apikey": os.getenv("OPENTRIMAP_API_KEY")}
    geo_resp = requests.get(geo_url, params=geo_params)
    geo_resp.raise_for_status()
    geo_data = geo_resp.json()
    lat = geo_data.get("lat")
    lon = geo_data.get("lon")
    if not lat or not lon:
        return {"error": "City not found, cannot get coordinates"}

    # Step 2: Get attractions by radius
    radius_url = "https://api.opentripmap.com/0.1/en/places/radius"
    radius_params = {
        "radius": 5000,   # 5km radius
        "lon": lon,
        "lat": lat,
        "limit": limit,
        "apikey": os.getenv("OPENTRIMAP_API_KEY"),
        "rate": 3,        # optional: filter by popularity
        "format": "json"
    }
    resp = requests.get(radius_url, params=radius_params)
    resp.raise_for_status()
    places = resp.json()

    activities = []
    for place in places:
        activities.append({
            "name": place.get("name"),
            "kinds": place.get("kinds"),
            "dist": place.get("dist")  # distance from city center
        })

    return activities

In [21]:
activities_agent = create_react_agent(
    model=llm,
    tools=[get_activities_opentripmap],
    prompt = (
    "You are a travel activity assistant. Your ONLY job is to use the get_activities tool "
    "to gather activity information before giving any answer. You MUST call the tool first "
    "with the exact parameters provided by the user.\n\n"
    "TOOL PARAMETERS:\n"
    "  • city: Name of the city (e.g., 'Paris')\n"
    "  • limit: Number of activities to retrieve (integer)\n\n"
    "STRICT RULES:\n"
    "1. Call the get_activities tool immediately with the user-provided parameters.\n"
    "2. Only after the tool call completes, you may provide output.\n"
    "3. If the tool returns empty results or fails, you may generate fallback data, "
    "but clearly note it is assumed.\n"
    "4. Do NOT answer or speculate before calling the tool.\n"
    "5. All output must be based on the tool results or fallback if unavailable."
)
,
    name="activities_agent",
)

In [22]:
for chunk in activities_agent.stream(
    {"messages": [{"role": "user", "content": "i want to go to bali from villupuram on 15th september.we are two adults and planning to stay for 3 daysin a luxurious place.suggest me relaxing activities"}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: activities_agent
Tool Calls:
  get_activities_opentripmap (fc_75cbcf08-e4b3-47e3-af66-0c04a7c045c2)
 Call ID: fc_75cbcf08-e4b3-47e3-af66-0c04a7c045c2
  Args:
    city: Bali
    limit: 5


Update from node tools:


================================= Tool Message =================================
Name: get_activities_opentripmap

[{"name": "Pura Maospait", "kinds": "religion,hindu_temples,interesting_places", "dist": 816.11365337}, {"name": "Museum Bali", "kinds": "cultural,museums,interesting_places,art_galleries", "dist": 862.12446531}, {"name": "Museum Agung Bung Karno", "kinds": "cultural,museums,interesting_places,other_museums", "dist": 2661.92559495}, {"name": "Gereja Katolik Roh Kudus Katedral", "kinds": "religion,cathedrals,interesting_places", "dist": 3014.1872893}, {"name": "Bajra Sandhi Monument", "kinds": "architecture,historic_architecture,interesting_places,other

In [23]:
from langgraph.types import Send
from typing import Annotated
from langchain_core.tools import tool, InjectedToolCallId
from langgraph.prebuilt import InjectedState

def create_task_description_handoff_tool(
    *, agent_name: str, description: str | None = None
):
    name = f"transfer_to_{agent_name}"
    description = description or f"Ask {agent_name} for help."

    @tool(name, description=description)
    def handoff_tool(
        # this is populated by the supervisor LLM
        task_description: Annotated[
            str,
            "Description of what the next agent should do, including all of the relevant context.",
        ],
        # these parameters are ignored by the LLM
        state: Annotated[MessagesState, InjectedState],
    ) -> Command:
        task_description_message = {"role": "user", "content": task_description}
        agent_input = {**state, "messages": [task_description_message]}
        return Command(
            goto=[Send(agent_name, agent_input)],
            graph=Command.PARENT,
        )

    return handoff_tool


assign_to_research_agent_with_description = create_task_description_handoff_tool(
    agent_name="research_agent",
    description="Assign task to a researcher agent.",
)

assign_to_weather_agent_with_description = create_task_description_handoff_tool(
    agent_name="weather_agent",
    description="Assign task to a weather agent.",
)

assign_to_flight_agent_with_description = create_task_description_handoff_tool(
    agent_name="flight_agent",
    description="Assign task to a flight agent.",
)

assign_to_hotel_agent_with_description = create_task_description_handoff_tool(
    agent_name="hotel_agent",
    description="Assign task to a hotel agent.",
)

assign_to_activities_agent_with_description = create_task_description_handoff_tool(
    agent_name="activities_agent",
    description="Assign task to a activities agent.",
)

In [24]:
supervisor = create_react_agent(
model=llm,
tools=[assign_to_research_agent_with_description, assign_to_weather_agent_with_description, assign_to_flight_agent_with_description, assign_to_hotel_agent_with_description, assign_to_activities_agent_with_description],
prompt=(
"You are a supervisor managing agents:\n"
"your only task is to use the agents and get the output and polish all the outputs from each agent's return and give it as a final itinerary\n"
"- a research agent. Find visa_info. Assign research-related tasks to this assistant and only output the tool's return.\n"
"- a weather agent. Assign weather_info-related tasks to this assistant and only output the tool's return.\n"
"- a flight agent. Assign flight_info-related tasks to this assistant and only output the tool's return.\n"
"- a hotel agent. Assign hotel_info-related tasks to this assistant and only output the tool's return.\n"
"- an activities agent. Assign activities_info-related tasks to this assistant and only output the tool's return.\n"
"Assign work to one agent at a time. Do not call agents in parallel.\n"
"Do not do any work yourself.\n"
"When visa,weather,flight,hotel,activities have been retrieved, output them concatenated and stop."
),
name="supervisor",
)
graph = (
StateGraph(MessagesState)
.add_node("supervisor", supervisor)
.add_node("research_agent", research_agent)
.add_node("weather_agent", weather_agent)
.add_node("flight_agent", flight_agent)
.add_node("hotel_agent", hotel_agent)
.add_node("activities_agent", activities_agent)
.add_edge(START, "supervisor")
.add_edge("supervisor", "research_agent")
.add_edge("supervisor", "weather_agent")
.add_edge("supervisor", "flight_agent")
.add_edge("supervisor", "hotel_agent")
.add_edge("supervisor", "activities_agent")
.add_edge("research_agent", "supervisor")
.add_edge("weather_agent", "supervisor")
.add_edge("flight_agent", "supervisor")
.add_edge("hotel_agent", "supervisor")
.add_edge("activities_agent", "supervisor")
.compile()
)


In [25]:
# supervisor_agent_with_description = create_react_agent(
#     model=llm,
#     tools=[
#         assign_to_research_agent_with_description,
#         assign_to_weather_agent_with_description,
#         assign_to_flight_agent_with_description,
#         assign_to_hotel_agent_with_description,
#         assign_to_activities_agent_with_description
#     ],
#     prompt=(
#         "You are a supervisor managing agents:\n"
#         "- a research agent.Find visa_info Assign research-related tasks to this assistant and only give the output given by the tool dont include info by yourself.\n"
#         "- a weather agent. Assign weather_info_related tasks to this assistant\n"
#         "- get the visa and weather details using the tools alone and output the gathered data from tools alone\n"
#         "Assign work to one agent at a time, do not call agents in parallel.\n"
#         "Do not do any work yourself."
#     ),
#     name="supervisor",
# )

# supervisor_with_description = (
#     StateGraph(MessagesState)
#     .add_node(
#         supervisor_agent_with_description, destinations=("research_agent", "weather_agent")
#     )
#     .add_node(research_agent)
#     .add_node(weather_agent)
#     .add_node(flight_agent)
#     .add_node(hotel_agent)
#     .add_node(activities_agent)
#     .add_edge(START, "supervisor")
#     .add_edge("research_agent", "supervisor")
#     .add_edge("weather_agent", "supervisor")
#     .add_edge("flight_agent", "supervisor")
#     .add_edge("hotel_agent", "supervisor")
#     .compile()
# )

In [26]:
# app = graph  # graph is already compiled above

# for chunk in app.stream(
#     {
#         "messages": [
#             {
#                 "role": "user",
#                 "content": "i want to go to bali from kerala ,we are 2 and looking to stay in comfy space then spend time relaxing for 2 days",
#             }
#         ]
#     }
# ):
#     # Only pass chunks that have "messages" to your printer
#     if isinstance(chunk, dict) and "messages" in chunk and chunk["messages"] is not None:
#         pretty_print_messages(chunk)
#     else:
#         # Optionally inspect non-message updates during debugging
#         print("fucked up")
        


In [4]:
@tool
def get_flight_offers(departure_city: str, arrival_city: str, travel_date: str) -> dict:
    """
    Fetch available flight offers from Amadeus API for a given route and date.

    Args:
        departure_city (str): IATA code of the departure city (e.g., "DEL").
        arrival_city (str): IATA code of the arrival city (e.g., "BOM").
        travel_date (str): Date of travel in YYYY-MM-DD format.

    Returns:
        dict: A dictionary with flight offers or an error message.
    """
    try:
        # 1. Get API credentials
        api_key = os.getenv("AMADEUS_API_KEY")
        api_secret = os.getenv("AMADEUS_API_SECRET")

        if not api_key or not api_secret:
            return {"error": "Missing API credentials in .env file"}

        # 2. Fetch access token
        token_url = "https://test.api.amadeus.com/v1/security/oauth2/token"
        payload = {
            "grant_type": "client_credentials",
            "client_id": api_key,
            "client_secret": api_secret,
        }
        token_resp = requests.post(token_url, data=payload)
        token_resp.raise_for_status()
        access_token = token_resp.json().get("access_token")

        if not access_token:
            return {"error": "Failed to retrieve access token"}

        # 3. Fetch flight offers
        url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
        headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json",
        }
        params = {
            "originLocationCode": departure_city,
            "destinationLocationCode": arrival_city,
            "departureDate": travel_date,
            "adults": 1,
            "max": 5,
        }
        resp = requests.get(url, headers=headers, params=params)
        resp.raise_for_status()

        return {
            "departure_city": departure_city,
            "arrival_city": arrival_city,
            "travel_date": travel_date,
            "offers": resp.json().get("data", []),
        }

    except requests.HTTPError as e:
        return {"error": f"HTTP Error: {e.response.text}"}
    except Exception as e:
        return {"error": str(e)}
    

In [5]:
res = get_flight_offers.func("DEL", "BOM", "2025-12-09")
print(res)


{'departure_city': 'DEL', 'arrival_city': 'BOM', 'travel_date': '2025-12-09', 'offers': [{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'isUpsellOffer': False, 'lastTicketingDate': '2025-08-30', 'lastTicketingDateTime': '2025-08-30', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT2H15M', 'segments': [{'departure': {'iataCode': 'DEL', 'terminal': '3', 'at': '2025-12-09T02:30:00'}, 'arrival': {'iataCode': 'BOM', 'terminal': '2', 'at': '2025-12-09T04:45:00'}, 'carrierCode': 'AI', 'number': '2983', 'aircraft': {'code': '32N'}, 'operating': {'carrierCode': 'AI'}, 'duration': 'PT2H15M', 'id': '1', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '84.18', 'base': '74.00', 'fees': [{'amount': '0.00', 'type': 'SUPPLIER'}, {'amount': '0.00', 'type': 'TICKETING'}], 'grandTotal': '84.18'}, 'pricingOptions': {'fareType': ['PUBLISHED'], 'includedCheckedBagsOnly': Tru

In [ ]:
%pip uninstall serpapi
%pip uninstall google-search-results

In [4]:


import os
from typing import Dict, Any, List
from serpapi import GoogleSearch
from langchain.tools import tool

# Load API key (make sure you set this in env)
SERPAPI_KEY = os.getenv("SERPAPI_KEY", "")


def fetch_flights(source: str, destination: str, departure_date: str, return_date: str = None) -> Dict[str, Any]:
    """
    Fetch flight data from SerpAPI Google Flights.
    """
    params = {
        "engine": "google_flights",
        "departure_id": source,          # e.g., "DEL"
        "arrival_id": destination,       # e.g., "BOM"
        "outbound_date": str(departure_date),
        "currency": "INR",
        "hl": "en",
        "api_key": SERPAPI_KEY
    }
    if return_date:
        params["return_date"] = str(return_date)

    search = GoogleSearch(params)
    return search.get_dict()


def process_flights(flight_data: Dict[str, Any], top_n: int = 5) -> List[Dict[str, Any]]:
    """
    Extract and sort flights by price, segregate by class.
    """
    best_flights = flight_data.get("best_flights", [])
    other_flights = flight_data.get("other_flights", [])

    all_flights = best_flights + other_flights

    # Sort flights by price
    sorted_flights = sorted(all_flights, key=lambda x: x.get("price", float("inf")))

    results = []
    for flight in sorted_flights[:top_n]:
        results.append({
            "airline": flight.get("airline_name"),
            "price": flight.get("price"),
            "duration": flight.get("duration"),
            "class": flight.get("class", "Economy"),   # default if not provided
            "link": flight.get("link")                # Google Flights deep link
        })
    return results


@tool("get_flights", return_direct=True)
def get_flights_tool(source: str, destination: str, departure_date: str, return_date: str = None) -> List[Dict[str, Any]]:
    """
    Get top cheapest flights between two cities on given dates.
    
    Args:
        source (str): Departure airport/city code (e.g., "DEL")
        destination (str): Arrival airport/city code (e.g., "BOM")
        departure_date (str): Departure date in YYYY-MM-DD
        return_date (str, optional): Return date in YYYY-MM-DD
    
    Returns:
        List[Dict]: Flights sorted by price with airline, price, class, duration, and link.
    """
    data = fetch_flights(source, destination, departure_date, return_date)
    return process_flights(data, top_n=5)


ImportError: cannot import name 'GoogleSearch' from 'serpapi' (d:\AGENTIC_AI\PROJECTS\Travel-Agent\venv\Lib\site-packages\serpapi\__init__.py)